In [37]:
import requests
import re
import os
import nltk
import bs4
import scipy
import numpy
import numpy.matlib
import random

import sys

# Download tagger and wordnet resources if you do not have them already
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
#%% Getting a list of directory contents
def gettextlist(directory_path):
    directory_textfiles=[]
    directory_nontextfiles=[]
    directory_nonfiles=[]
    # Process each item in the directory
    directory_contents=os.listdir(directory_path)
    for contentitem in directory_contents:
        temp_fullpath=os.path.join(directory_path, contentitem)
        # Non-files (e.g. subdirectories) are stored separately
        if os.path.isfile(temp_fullpath)==0:
            directory_nonfiles.append(contentitem)
        else:
            # Is this a non-text file (not ending in .txt)?
            if temp_fullpath.find('.txt')==-1:
                directory_nontextfiles.append(contentitem)
            else:
                # This is a text file
                directory_textfiles.append(contentitem)
    return(directory_textfiles,directory_nontextfiles,directory_nonfiles)

In [39]:
# %% Basic file crawler
def basicfilecrawler(directory_path):
    # Store filenames read and their text content
    num_files_read = 0
    crawled_filenames = []
    crawled_texts = []
    directory_contentlists = gettextlist(directory_path)
    # In this basic crawled we just process text files
    # and do not handle subdirectories
    # directory_textfiles=directory_contentlists[0]
    directory_textfiles = directory_contentlists[1]
    for contentitem in directory_textfiles:
        # print('Reading file:')
        # print(contentitem)
        # Open the file and read its contents
        temp_fullpath = os.path.join(directory_path, contentitem)
        temp_file = open(temp_fullpath, "r", encoding="utf-8", errors="ignore")
        temp_text = temp_file.read()
        temp_file.close()
        # Store the read filename and content
        crawled_filenames.append(contentitem)
        crawled_texts.append(temp_text)
        num_files_read = num_files_read + 1
    return (crawled_filenames, crawled_texts)

In [40]:
def read_folder(folder):
    # Read the folder contents and return the list of folder names (not including the subfolders)
    # and the list of file names
    folder_names=[]
    for root, dirs, files in os.walk(folder):
        for name in dirs:
            folder_names.append(os.path.join(root, name))
    return folder_names

In [41]:
# Read the folder contents
file_list = read_folder(
    "20_newsgroups"
)
print(file_list)

['20_newsgroups\\alt.atheism', '20_newsgroups\\comp.graphics', '20_newsgroups\\comp.os.ms-windows.misc', '20_newsgroups\\comp.sys.ibm.pc.hardware', '20_newsgroups\\comp.sys.mac.hardware', '20_newsgroups\\comp.windows.x', '20_newsgroups\\misc.forsale', '20_newsgroups\\rec.autos', '20_newsgroups\\rec.motorcycles', '20_newsgroups\\rec.sport.baseball', '20_newsgroups\\rec.sport.hockey', '20_newsgroups\\sci.crypt', '20_newsgroups\\sci.electronics', '20_newsgroups\\sci.med', '20_newsgroups\\sci.space', '20_newsgroups\\soc.religion.christian', '20_newsgroups\\talk.politics.guns', '20_newsgroups\\talk.politics.mideast', '20_newsgroups\\talk.politics.misc', '20_newsgroups\\talk.religion.misc']


In [42]:
# Create a list of the folders to be read
folders_to_read = [
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
]

# Find the indices of the folders to be read in file_list and store them in a list
indices_to_read = []
for i in range(len(file_list)):
    # Check if the folder name is in the list of folders to be read (match the last part of the path)
    if file_list[i].split("\\")[-1] in folders_to_read:
        indices_to_read.append(i)

print(indices_to_read)

print(file_list[indices_to_read[0]])
rec_autos_filenames, rec_autos_texts = basicfilecrawler(file_list[indices_to_read[0]])
rec_motorcycles_filenames, rec_motorcycles_texts = basicfilecrawler(
    file_list[indices_to_read[1]]
)
rec_sport_baseball_filenames, rec_sport_baseball_texts = basicfilecrawler(
    file_list[indices_to_read[2]]
)
rec_sport_hockey_filenames, rec_sport_hockey_texts = basicfilecrawler(
    file_list[indices_to_read[3]]
)

[7, 8, 9, 10]
20_newsgroups\rec.autos


In [43]:
print(rec_autos_filenames[0:5])
print(rec_autos_texts[0:5])
print(rec_motorcycles_filenames[0:5])
print(rec_motorcycles_texts[0:5])
print(rec_sport_baseball_filenames[0:5])
print(rec_sport_baseball_texts[0:5])
print(rec_sport_hockey_filenames[0:5])
print(rec_sport_hockey_texts[0:5])

['101551', '101552', '101553', '101554', '101555']
["Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!wupost!uunet!brunix!cs.brown.edu!cs012055\nFrom: cs012055@cs.brown.edu (Hok-Chung Tsang)\nNewsgroups: rec.autos\nSubject: Re: Saturn's Pricing Policy\nMessage-ID: <1993Apr5.230808.581@cs.brown.edu>\nDate: 5 Apr 93 23:08:08 GMT\nArticle-I.D.: cs.1993Apr5.230808.581\nReferences: <C4oxwp.KKM@news.cso.uiuc.edu> <C4vIr5.L3r@shuksan.ds.boeing.com>\nSender: news@cs.brown.edu\nOrganization: Brown Computer Science Dept.\nLines: 51\n\nIn article <C4vIr5.L3r@shuksan.ds.boeing.com>, fredd@shuksan (Fred Dickey) writes:\n|> CarolinaFan@uiuc (cka52397@uxa.cso.uiuc.edu) wrote:\n|> : \tI have been active in defending Saturn lately on the net and would\n|> : like to state my full opinion on the subject, rather than just reply to others'\n|> : points.\n|> : \t\n|> : \tThe biggest problem some people seem to be having is that Saturn\n|> : Dealers make ~$2K on a car.  I think most will a

In [44]:
mycrawled_texts = (
    rec_autos_texts
    + rec_motorcycles_texts
    + rec_sport_baseball_texts
    + rec_sport_hockey_texts
)
mycrawled_filenames = (
    rec_autos_filenames
    + rec_motorcycles_filenames
    + rec_sport_baseball_filenames
    + rec_sport_hockey_filenames
)

In [45]:
# Exclude header lines from each message
excludedlinemarkers = [
    "Xref:",
    "Path:",
    "From:",
    "Newsgroups:",
    "Subject:",
    "Summary:",
    "Keywords:",
    "Message-ID:",
    "Date:",
    "Expires:",
    "Followup-To:",
    "Distribution:",
    "Organization:",
    "Approved:",
    "Supersedes:",
    "Lines:",
    "NNTP-Posting-Host:",
    "References:",
    "Sender:",
    "In-Reply-To:",
    "Article-I.D.:",
    "Reply-To:",
    "Nntp-Posting-Host:",
]
for k in range(len(mycrawled_texts)):
    templines = mycrawled_texts[k].splitlines()
    remaininglines = []
    for l in range(len(templines)):
        line_should_be_excluded = 0
        for m in range(len(excludedlinemarkers)):
            if len(templines[l]) >= len(excludedlinemarkers[m]):
                if (
                    excludedlinemarkers[m]
                    == templines[l][0 : len(excludedlinemarkers[m])]
                ):
                    line_should_be_excluded = 1
                    break
        if line_should_be_excluded == 0:
            remaininglines.append(templines[l])
    mycrawled_texts[k] = "\n".join(remaininglines)

In [46]:
# %% Tokenize loaded texts and change them to NLTK format
mycrawled_nltktexts = []
for k in range(len(mycrawled_texts)):
    temp_tokenizedtext = nltk.word_tokenize(mycrawled_texts[k])
    temp_nltktext = nltk.Text(temp_tokenizedtext)
    mycrawled_nltktexts.append(temp_nltktext)

In [47]:
# %% Make all crawled texts lowercase
mycrawled_lowercasetexts = []
for k in range(len(mycrawled_nltktexts)):
    temp_lowercasetext = []
    for l in range(len(mycrawled_nltktexts[k])):
        lowercaseword = mycrawled_nltktexts[k][l].lower()
        temp_lowercasetext.append(lowercaseword)
    temp_lowercasetest = nltk.Text(temp_lowercasetext)
    mycrawled_lowercasetexts.append(temp_lowercasetext)

In [48]:
# %% Convert a POS tag for WordNet
def tagtowordnet(postag):
    wordnettag = -1
    if postag[0] == "N":
        wordnettag = "n"
    elif postag[0] == "V":
        wordnettag = "v"
    elif postag[0] == "J":
        wordnettag = "a"
    elif postag[0] == "R":
        wordnettag = "r"
    return wordnettag

In [49]:
# %% POS tag and lemmatize the loaded texts

lemmatizer = nltk.stem.WordNetLemmatizer()


def lemmatizetext(nltktexttolemmatize):
    # Tag the text with POS tags
    taggedtext = nltk.pos_tag(nltktexttolemmatize)
    # Lemmatize each word text
    lemmatizedtext = []
    for l in range(len(taggedtext)):
        # Lemmatize a word using the WordNet converted POS tag
        wordtolemmatize = taggedtext[l][0]
        wordnettag = tagtowordnet(taggedtext[l][1])
        if wordnettag != -1:
            lemmatizedword = lemmatizer.lemmatize(wordtolemmatize, wordnettag)
        else:
            lemmatizedword = wordtolemmatize
        # Store the lemmatized word
        lemmatizedtext.append(lemmatizedword)
    return lemmatizedtext


mycrawled_lemmatizedtexts = []
for k in range(len(mycrawled_lowercasetexts)):
    lemmatizedtext = lemmatizetext(mycrawled_lowercasetexts[k])
    lemmatizedtext = nltk.Text(lemmatizedtext)
    mycrawled_lemmatizedtexts.append(lemmatizedtext)

In [50]:
# %% Find the vocabulary, in a distributed fashion
myvocabularies = []
myindices_in_vocabularies = []
# Find the vocabulary of each document
for k in range(len(mycrawled_lemmatizedtexts)):
    # Get unique words and where they occur
    temptext = mycrawled_lemmatizedtexts[k]
    uniqueresults = numpy.unique(temptext, return_inverse=True)
    uniquewords = uniqueresults[0]
    wordindices = uniqueresults[1]
    # Store the vocabulary and indices of document words in it
    myvocabularies.append(uniquewords)
    myindices_in_vocabularies.append(wordindices)

In [51]:
# Unify the vocabularies.
# First concatenate all vocabularies
tempvocabulary = []
for k in range(len(mycrawled_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])
# Find the unique elements among all vocabularies
uniqueresults = numpy.unique(tempvocabulary, return_inverse=True)
unifiedvocabulary = uniqueresults[0]
wordindices = uniqueresults[1]
# Translate previous indices to the unified vocabulary.
# Must keep track where each vocabulary started in
# the concatenated one.
vocabularystart = 0
myindices_in_unifiedvocabulary = []
for k in range(len(mycrawled_lemmatizedtexts)):
    # In order to shift word indices, we must temporarily
    # change their data type to a Numpy array
    tempindices = numpy.array(myindices_in_vocabularies[k])
    tempindices = tempindices + vocabularystart
    tempindices = wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart = vocabularystart + len(myvocabularies[k])

In [52]:
# %% Count the numbers of occurrences of each unique word
# Let's count also various statistics over the documents
unifiedvocabulary_totaloccurrencecounts = numpy.zeros((len(unifiedvocabulary), 1))
unifiedvocabulary_documentcounts = numpy.zeros((len(unifiedvocabulary), 1))
unifiedvocabulary_meancounts = numpy.zeros((len(unifiedvocabulary), 1))
unifiedvocabulary_countvariances = numpy.zeros((len(unifiedvocabulary), 1))

In [53]:
# First pass: count occurrences
for k in range(len(mycrawled_lemmatizedtexts)):
    occurrencecounts = numpy.zeros((len(unifiedvocabulary), 1))
    for l in range(len(myindices_in_unifiedvocabulary[k])):
        occurrencecounts[myindices_in_unifiedvocabulary[k][l]] = (
            occurrencecounts[myindices_in_unifiedvocabulary[k][l]] + 1
        )
    unifiedvocabulary_totaloccurrencecounts = (
        unifiedvocabulary_totaloccurrencecounts + occurrencecounts
    )
    unifiedvocabulary_documentcounts = unifiedvocabulary_documentcounts + (
        occurrencecounts > 0
    )

In [54]:
# %% Inspect frequent words
# Sort words by largest total (or mean) occurrence count
highest_totaloccurrences_indices = numpy.argsort(
    -1 * unifiedvocabulary_totaloccurrencecounts, axis=0
)
print(numpy.squeeze(unifiedvocabulary[highest_totaloccurrences_indices[0:100]]))
print(
    numpy.squeeze(
        unifiedvocabulary_totaloccurrencecounts[highest_totaloccurrences_indices[0:100]]
    )
)

['>' 'the' ',' '.' '--' 'be' '*' 'a' 'to' 'i' ')' 'in' 'and' '(' 'of' ':'
 '@' 'have' 'that' 'it' 'you' '!' 'for' 'do' '?' 'on' '|' "'s" '-' "n't"
 'this' "''" 'with' 'but' 'he' 'not' '0' 'they' '1' 'as' '<' 'at' 'if'
 'get' '``' 'write' 'or' 'my' '...' '2' 'article' 'game' 'go' 'would' '#'
 'can' 'about' 'one' 'will' 'all' 'there' 'an' 'what' 'from' 'out' 'so'
 'good' 'car' 'like' 'year' 'think' 'by' 'me' 'team' 'up' 'his' '%' 'just'
 'when' '3' 'more' 'make' 'your' 'no' 'know' 'any' 'say' 'who' 'than' '4'
 'we' 'time' 'some' 'play' 'well' 'see' 'how' 'player' 'win' 'only']
[40640. 40032. 39497. 39110. 35859. 27296. 20092. 17910. 17358. 15952.
 14883. 14110. 14036. 13692. 13180. 11444.  9901.  9477.  9199.  8716.
  7144.  7079.  7017.  6890.  6686.  6048.  6030.  5907.  4914.  4911.
  4768.  4515.  4514.  4288.  4140.  4041.  3977.  3925.  3803.  3753.
  3648.  3569.  3533.  3267.  3265.  3261.  3241.  3225.  2855.  2854.
  2745.  2728.  2696.  2671.  2518.  2416.  2392.  2376.  2314.

In [55]:
#%% Vocabulary pruning
nltkstopwords=nltk.corpus.stopwords.words('english')
pruningdecisions=numpy.zeros((len(unifiedvocabulary),1))
for k in range(len(unifiedvocabulary)):
    # Rule 1: check the nltk stop word list
    if (unifiedvocabulary[k] in nltkstopwords):
        pruningdecisions[k]=1
    # Rule 2: if the word is in the top 1% of frequent words
    if (k in highest_totaloccurrences_indices[\
        0:int(numpy.floor(len(unifiedvocabulary)*0.01))]):
        pruningdecisions[k]=1
    # Rule 3: if the word is too short
    if len(unifiedvocabulary[k])<3:
        pruningdecisions[k]=1
    # Rule 4: if the word is too long
    if len(unifiedvocabulary[k])>20:
        pruningdecisions[k]=1
    # Rule 5: if the word has unwanted characters
    # (here for simplicity only a-z allowed)
    # if unifiedvocabulary[k].isalpha()==False:
    #     pruningdecisions[k]=1
    # Rule 7: if the word occurs less than 4 times
    if(unifiedvocabulary_totaloccurrencecounts[k] < 4):
        pruningdecisions[k] = 1

In [56]:
#%% Inspect remaining frequent words
# Sort remaining words by largest total (or mean) occurrence count
remainingindices=numpy.squeeze(numpy.where(pruningdecisions==0)[0])
remainingvocabulary=unifiedvocabulary[remainingindices]
remainingvocabulary_totaloccurrencecounts= \
    unifiedvocabulary_totaloccurrencecounts[remainingindices]
remaining_highest_totaloccurrences_indices= \
    numpy.argsort(-1*remainingvocabulary_totaloccurrencecounts,axis=0)
print(numpy.squeeze(remainingvocabulary[remaining_highest_totaloccurrences_indices[0:100]]))
print(numpy.squeeze(remainingvocabulary_totaloccurrencecounts[ \
    remaining_highest_totaloccurrences_indices[0:100]]))

['wheel' 'yet' 'driver' 'live' 'model' 'able' 'bit' 'ago' 'city' 'cost'
 'total' 'mind' 'mention' 'man' 'brave' 'ford' 'penalty' 'final' 'large'
 'anyway' 'bill' 'idea' 'coach' 'saw' 'performance' 'beat' 'version'
 'montreal' 'rule' 'group' 'ice' 'seat' 'face' 'hitter' 'friend'
 'although' 'almost' 'today' 'order' 'comment' 'bos' 'minute' 'stats'
 'hold' 'det' 'follow' 'american' 'insurance' 'job' 'month' 'walk' 'local'
 'past' 'smith' 'joe' 'honda' 'easy' 'tie' 'cal' 'news' 'hell' 'ticket'
 'x-newsreader' 'wait' 'bring' 'hope' 'jet' 'helmet' 'stuff' 'morris'
 'rider' 'left' 'word' 'compare' 'van' 'note' 'experience' 'york' 'center'
 'information' 'flyer' 'add' 'rear' 'pen' 'puck' 'contact' 'set' 'defense'
 'young' 'break' 'netcom.com' 'others' 'claim' 'later' 'e-mail' 'design'
 'matter' 'late' 'whether' 'tor']
[254. 253. 251. 251. 250. 249. 248. 247. 246. 246. 245. 245. 244. 244.
 243. 241. 240. 240. 239. 237. 232. 232. 230. 229. 229. 228. 225. 224.
 223. 222. 222. 222. 221. 221. 221.

In [57]:
# %% Get indices of documents to remaining words
oldtopruned = []
tempind = -1
for k in range(len(unifiedvocabulary)):
    if pruningdecisions[k] == 0:
        tempind = tempind + 1
        oldtopruned.append(tempind)
    else:
        oldtopruned.append(-1)

# %% Create pruned texts

mycrawled_prunedtexts = []
myindices_in_prunedvocabulary = []
for k in range(len(mycrawled_lemmatizedtexts)):
    temp_newindices = []
    temp_newdoc = []
    for l in range(len(mycrawled_lemmatizedtexts[k])):
        temp_oldindex = myindices_in_unifiedvocabulary[k][l]
        temp_newindex = oldtopruned[temp_oldindex]
        if temp_newindex != -1:
            temp_newindices.append(temp_newindex)
            temp_newdoc.append(unifiedvocabulary[temp_oldindex])
    mycrawled_prunedtexts.append(temp_newdoc)
    myindices_in_prunedvocabulary.append(temp_newindices)

Exercise 12.1: Paragraph embedding of four newsgroups.

Using the same data as in exercise 6.1 (four newsgroups), use paragraph embedding to create a
vector for each document, using the Python implementation discussed on the lecture (lecture 12,
slides 12-13), or using a different language/library of your choice. 

Then find, by Euclidean distance, the closest other documents for the following documents: 101551
(part of rec.autos), 103118 (part of rec.motorcycles), 98657 (part of rec.sport.baseball) and 52550
(part of rec.sport.hockey). Are the closest documents from the same newsgroup? Does their content
seem to match that of the original document?
Report your code, results, and discussion.

In [58]:
import gensim
# We need to create a tagged version of each document
gensim_tagged_docs=[]
gensim_docs=mycrawled_prunedtexts
for k in range(len(mycrawled_prunedtexts)):
    doctag='doc'+str(k)
    tagged_document= \
        gensim.models.doc2vec.TaggedDocument( \
        mycrawled_prunedtexts[k],[doctag])
    gensim_tagged_docs.append(tagged_document)
# Create a dictionary from the documents
gensim_dictionary = gensim.corpora.Dictionary(gensim_docs)

In [59]:
# Train the word2vec model
# The dm_concat parameter controls whether to concatenate 
# or average word vectors when learning the paragraph 
# vector  (see slides 5 and 6).
doc2vecmodel = gensim.models.doc2vec.Doc2Vec(gensim_tagged_docs, \
    vector_size=10, window=5, min_count=1, \
    workers=4, dm_concat=0)

In [60]:
# 101551 (part of rec.autos), 103118 (part of rec.motorcycles), 98657 (part of rec.sport.baseball) and 52550
# (part of rec.sport.hockey)
# find the indices of the documents in the original list (mycrawled_filenames)
# NOTE: I have modified the 103118 version in rec.auto into 103816
docindices=[]
for k in range(len(mycrawled_filenames)):
    if mycrawled_filenames[k].find('101551')!=-1:
        docindices.append(k)
    if mycrawled_filenames[k].find('103118')!=-1:
        print(mycrawled_filenames[k])
        docindices.append(k)
    if mycrawled_filenames[k].find('98657')!=-1:
        docindices.append(k)
    if mycrawled_filenames[k].find('52550')!=-1:
        docindices.append(k)
print(docindices)

103118
[0, 1003, 2998, 3000]


In [64]:
from scipy.spatial.distance import euclidean
dist_101551_103118 = euclidean(doc2vecmodel['doc0'], doc2vecmodel['doc1003'])
dist_101551_98657 = euclidean(doc2vecmodel['doc0'], doc2vecmodel['doc2998'])
dist_101551_52550 = euclidean(doc2vecmodel['doc0'], doc2vecmodel['doc3000'])

dist_103118_98657 = euclidean(doc2vecmodel['doc1003'], doc2vecmodel['doc2998'])
dist_103118_52550 = euclidean(doc2vecmodel['doc1003'], doc2vecmodel['doc3000']) 

dist_98657_52550 = euclidean(doc2vecmodel['doc2998'], doc2vecmodel['doc3000']) 

In [65]:
print(f"The distance between 101551 and 103118 is {dist_101551_103118}")
print(f"The distance between 101551 and 98657 is {dist_101551_98657}")
print(f"The distance between 101551 and 52550 is {dist_101551_52550}")
print()
print(f"The distance between 103118 and 98657 is {dist_103118_98657}")
print(f"The distance between 103118 and 52550 is {dist_103118_52550}")
print()
print(f"The distance between 98657 and 52550 is {dist_98657_52550}")

The distance between 101551 and 103118 is 0.8738444447517395
The distance between 101551 and 98657 is 1.2525023221969604
The distance between 101551 and 52550 is 1.474739670753479

The distance between 103118 and 98657 is 1.5823026895523071
The distance between 103118 and 52550 is 1.9398778676986694

The distance between 98657 and 52550 is 1.0018060207366943


The closest document and the farthest document could be seen in the code above

Their content seems to match that of the original document